### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [11]:
# Calculate the total number of schools 
total_number_schools = len(school_data_complete["School ID"].unique())

# Calculate the total number of students 
total_number_students = len(school_data_complete["Student ID"].unique())

# Calculate the total budget
total_budget = school_data["budget"].sum()

# Calculate the average math score
average_math = student_data["math_score"].mean()

# Calculate the average reading score
average_reading = student_data["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
student_data["passing_math"] = student_data["math_score"] >= 70
percent_passing_math = (student_data["passing_math"].mean())*100

# Calculate the percentage of students with a passing reading score (70 or greater)
student_data["passing_reading"] = student_data["reading_score"] >= 70
percent_passing_reading = (student_data["passing_reading"].mean())*100

# Calculate the percentage of students who passed math and reading (% Overall Passing)
student_data["overall_passing"] = student_data["passing_math"] & student_data["passing_reading"]
percent_overall_passing = (student_data["overall_passing"].mean())*100

# Create a dataframe to hold the above results
district_results = [{"Total Schools": total_number_schools, "Total Students": total_number_students, 
                    "Total Budget": total_budget, "Average Math Score": average_math,
                    "Average Reading Score": average_reading, "% Passing Math": percent_passing_math,
                    "% Passing Reading": percent_passing_reading, "% Overall Passing": percent_overall_passing}]
district_summary_table = pd.DataFrame(district_results)

# Give the displayed data cleaner formatting
district_summary_table["Total Students"] = district_summary_table["Total Students"].map("{:,}".format)
district_summary_table["Total Budget"] = district_summary_table["Total Budget"].map("${:,.2f}".format)
district_summary_table["Average Math Score"] = district_summary_table["Average Math Score"].map("{:,.2f}".format)
district_summary_table["Average Reading Score"] = district_summary_table["Average Reading Score"].map("{:,.2f}".format)
district_summary_table["% Passing Math"] = district_summary_table["% Passing Math"].map("{:,.2f}%".format)
district_summary_table["% Passing Reading"] = district_summary_table["% Passing Reading"].map("{:,.2f}%".format)
district_summary_table["% Overall Passing"] = district_summary_table["% Overall Passing"].map("{:,.2f}%".format)
district_summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [12]:
# Group by School Name 
school_group = school_data_complete.groupby(["school_name"]).mean()

# Calculate Per Student Budget
school_group["Per Student Budget"] = school_group["budget"]/school_group["size"]

# Calculate Students Per School to be used in percentage calculations
school_names = school_data_complete.set_index("school_name").groupby(["school_name"])
students_per_school = school_names["Student ID"].count()

#Calculate % Passing Math
school_group["% Passing Math"] = (school_data_complete[school_data_complete["math_score"] >=70].groupby("school_name")["Student ID"].count()
             /students_per_school)*100

#Calculate % Passing Reading 
school_group["% Passing Reading"] = (school_data_complete[school_data_complete["reading_score"] >=70].groupby("school_name")["Student ID"].count()
             /students_per_school)*100

#Calculate % Overall Passing
school_group["% Overall Passing"] = (school_data_complete[(school_data_complete["math_score"] >=70) & (school_data_complete["reading_score"] >=70)].groupby("school_name")["Student ID"].count()
                   /students_per_school)*100

# Merge the two datas together in order to be have school_name as a column 
school_data_summary = pd.merge(school_group, school_data, how="left", on=["school_name", "school_name"])

# To Delete Unnecessary Columns
del school_data_summary['size_y']
del school_data_summary['budget_y']
del school_data_summary['Student ID']
del school_data_summary['School ID_x']
del school_data_summary['School ID_y']

#Create a dataframe to store the results
School_results = pd.DataFrame({"School Name":  school_data_summary["school_name"],
                                "School Type": school_data_summary["type"],
                               "Total Students":school_data_summary["size_x"],
                               "Total School Budget": school_data_summary["budget_x"],
                               "Per Student Budget":school_data_summary["Per Student Budget"], 
                               "Average Math Score":round(school_data_summary["math_score"],2),
                               "Average Reading Score":round(school_data_summary["reading_score"],2), 
                               "% Passing Math": school_data_summary["% Passing Math"],
                               "% Passing Reading": school_data_summary["% Passing Reading"],
                               "% Overall Passing": school_data_summary["% Overall Passing"]}) 

# For Cleaner Formatting
School_results["Total Students"] = School_results["Total Students"].map("{:,}".format)
School_results["Total School Budget"] = School_results["Total School Budget"].map("${:,.2f}".format)
School_results["Per Student Budget"] = School_results["Per Student Budget"].map("${:,.2f}".format)
School_results["% Passing Math"] = School_results["% Passing Math"].map("{:,.2f}%".format)
School_results["% Passing Reading"] = School_results["% Passing Reading"].map("{:,.2f}%".format)
School_results["% Overall Passing"] = School_results["% Overall Passing"].map("{:,.2f}%".format)
School_results

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,"4,976.0","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
1,Cabrera High School,Charter,"1,858.0","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
2,Figueroa High School,District,"2,949.0","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
3,Ford High School,District,"2,739.0","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
4,Griffin High School,Charter,"1,468.0","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
5,Hernandez High School,District,"4,635.0","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
6,Holden High School,Charter,427.0,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
7,Huang High School,District,"2,917.0","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
8,Johnson High School,District,"4,761.0","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
9,Pena High School,Charter,962.0,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [13]:
top_5= School_results.sort_values("% Overall Passing", ascending = False)
top_5.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,"1,858.0","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
12,Thomas High School,Charter,"1,635.0","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
4,Griffin High School,Charter,"1,468.0","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
13,Wilson High School,Charter,"2,283.0","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
9,Pena High School,Charter,962.0,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [14]:
bottom_5= School_results.sort_values("% Overall Passing")
bottom_5.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,"3,999.0","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
2,Figueroa High School,District,"2,949.0","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
7,Huang High School,District,"2,917.0","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
5,Hernandez High School,District,"4,635.0","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
8,Johnson High School,District,"4,761.0","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:
# Lists the average Math Score for 9th 
ninth = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["math_score"].mean()

# Lists the average Math Score for 10th 
tenth = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["math_score"].mean()

# Lists the average Math Score for 11th
eleventh = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["math_score"].mean()

# Lists the average Math Score for 12th 
twelfth = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["math_score"].mean()

# Combine the series into a DataFrame
math_scores = pd.DataFrame({"9th": ninth, "10th": tenth, "11th": eleventh, "12th": twelfth})
math_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [16]:
# Lists the average Reading Score for 9th 
ninth_read = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["reading_score"].mean()

# Lists the average Reading Score for 10th 
tenth_read = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["reading_score"].mean()

# Lists the average Reading Score for 11th 
eleventh_read = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["reading_score"].mean()

# Lists the average Reading Score for 12th 
twelfth_read = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

# Combine the series into a DataFrame
reading_scores = pd.DataFrame({"9th": ninth_read, "10th": tenth_read, "11th": eleventh_read, "12th": twelfth_read})
reading_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
# Sample bins
spending_bins = [0, 585, 630, 645, 675]
group_names = ["< $584", "$585-630", "$615-645", "$645-675"]

# Use 4 reasonable bins to group school spending. 
school_data_summary["Spending Ranges"] = pd.cut(school_data_summary["Per Student Budget"], spending_bins, labels=group_names, right = False)

# Group by Spending Ranges
spending_grouped = school_data_summary.groupby("Spending Ranges").mean() 

# Delete Unnecessary Columns
del spending_grouped['size_x']
del spending_grouped['budget_x']
del spending_grouped['Per Student Budget']

# Reorder Columns
organized_spending_grouped= spending_grouped[["math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
organized_spending_grouped

# Rename Columns
renamed_organized_spending_grouped= organized_spending_grouped.rename(columns={"math_score": "Average Math Score", "reading_score": "Average Reading Score"})

# Format
renamed_organized_spending_grouped["Average Math Score"] = renamed_organized_spending_grouped["Average Math Score"].map("{:.2f}".format)
renamed_organized_spending_grouped["Average Reading Score"] = renamed_organized_spending_grouped["Average Reading Score"].map("{:.2f}".format)
renamed_organized_spending_grouped["% Passing Math"] = renamed_organized_spending_grouped["% Passing Math"].map("{:,.2f}%".format)
renamed_organized_spending_grouped["% Passing Reading"] = renamed_organized_spending_grouped["% Passing Reading"].map("{:,.2f}%".format)
renamed_organized_spending_grouped["% Overall Passing"] = renamed_organized_spending_grouped["% Overall Passing"].map("{:,.2f}%".format)

renamed_organized_spending_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges,,,,,
< $584,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$615-645,78.52,81.62,73.48%,84.39%,62.86%
$645-675,77.00,81.03,66.16%,81.13%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [82]:
# Sample bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (< 1000)", "Medium(1000-2000)", "Large (2000-3000)"]

# Use 4 reasonable bins to group school spending. 
school_data_summary["School Size"] = pd.cut(school_data_summary["size_x"], size_bins, labels=group_names)

# Group by School Size
size_grouped = school_data_summary.groupby("School Size").mean() 

# Delete Unncessary Columns
del size_grouped['size_x']
del size_grouped['budget_x']
del size_grouped['Per Student Budget']

# Reorder Columns
organized_size_grouped= size_grouped[["math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
organized_size_grouped

# Rename Columns
renamed_organized_size_grouped= organized_size_grouped.rename(columns={"math_score": "Average Math Score", "reading_score": "Average Reading Score"})

# Format 
renamed_organized_size_grouped["Average Math Score"] = renamed_organized_size_grouped["Average Math Score"].map("{:.2f}".format)
renamed_organized_size_grouped["Average Reading Score"] = renamed_organized_size_grouped["Average Reading Score"].map("{:.2f}".format)
renamed_organized_size_grouped["% Passing Math"] = renamed_organized_size_grouped["% Passing Math"].map("{:,.2f}%".format)
renamed_organized_size_grouped["% Passing Reading"] = renamed_organized_size_grouped["% Passing Reading"].map("{:,.2f}%".format)
renamed_organized_size_grouped["% Overall Passing"] = renamed_organized_size_grouped["% Overall Passing"].map("{:,.2f}%".format)

renamed_organized_size_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (< 1000),83.82,83.93,93.55%,96.10%,89.88%
Medium(1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-3000),77.75,81.34,69.96%,82.77%,58.29%


## Scores by School Type

* Perform the same operations as above, based on school type

In [83]:
# Group by Type of School
school_type_grouped = school_data_summary.groupby("type").mean()

# Delete Unncessary Columns
del school_type_grouped['size_x']
del school_type_grouped['budget_x']
del school_type_grouped['Per Student Budget']

# Reorder Columns
organized_school_type_grouped= school_type_grouped[["math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
organized_school_type_grouped

# Rename Columns
renamed_organized_school_type_grouped= organized_school_type_grouped.rename(columns={"math_score": "Average Math Score", "reading_score": "Average Reading Score"})

# Format
renamed_organized_school_type_grouped["Average Math Score"] = renamed_organized_school_type_grouped["Average Math Score"].map("{:.2f}".format)
renamed_organized_school_type_grouped["Average Reading Score"] = renamed_organized_school_type_grouped["Average Reading Score"].map("{:.2f}".format)
renamed_organized_school_type_grouped["% Passing Math"] = renamed_organized_school_type_grouped["% Passing Math"].map("{:,.2f}%".format)
renamed_organized_school_type_grouped["% Passing Reading"] = renamed_organized_school_type_grouped["% Passing Reading"].map("{:,.2f}%".format)
renamed_organized_school_type_grouped["% Overall Passing"] = renamed_organized_school_type_grouped["% Overall Passing"].map("{:,.2f}%".format)

renamed_organized_school_type_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
